# Project 

## Team Members
Eldin Basic and Siranush Mazmandyan

## Project Code

In [13]:
import games4e
from games4e import Game, minmax_decision, GameState, alpha_beta_search
from collections import namedtuple
from copy import deepcopy
import time

In [48]:
import random
random.seed(20)

In [49]:
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

In [61]:
import random

class Mancala(Game):
    def __init__(self, pits_per_player=6, stones_per_pit=4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player - 1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player + 1, len(self.board) - 2]
        self.p2_mancala_index = len(self.board) - 1

        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn) 

    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        if self.current_player == 1:
            if ((self.p1_pits_index[0] <= pit-1) and (pit-1 <= self.p1_pits_index[1]) and self.board[pit-1] > 0):
                self.moves.append((self.current_player, pit))
                return True
            else:
                return False
        else: 
            if ((self.p2_pits_index[0] <= pit+self.pits_per_player-1) and 
                (pit+self.pits_per_player-1 <= self.p2_pits_index[1]) and self.board[pit+self.pits_per_player-1] > 0):
                self.moves.append((self.current_player, pit))
                return True
            else:
                return False

    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        # write your code here
        if self.current_player == 1:
            pits = range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)
        else:
            pits = range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)

        valid_pits = [i + 1 if self.current_player == 1 else i - self.pits_per_player + 1
                  for i in pits if self.board[i] > 0]

        return random.choice(valid_pits) if valid_pits else None

    def play(self, pit):
        """
         This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
         1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
         2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
         3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

         Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        player = self.current_player
        if(pit > self.pits_per_player):
            pit = pit - self.pits_per_player
        #print(f"Player {player} selected pit {pit}")
        if not self.valid_move(pit):
            #print("INVALID MOVE")
            # self.current_player = 3 - self.current_player
            return self.board
        
        if not self.is_terminal():
            if self.current_player == 2:
                pit += self.pits_per_player
            else:  
                pit -= 1
            stones = self.board[pit]
            self.board[pit] = 0
            index = pit

            while stones > 0:
                index = (index + 1) % len(self.board)
                if ((self.current_player == 1 and index == self.p2_mancala_index) or 
                   (self.current_player == 2 and index == self.p1_mancala_index)):
                    continue
                else:
                    self.board[index] += 1
                    stones -= 1

            if ((self.current_player == 1 and self.p1_pits_index[0] <= index <= self.p1_pits_index[1] and self.board[index] == 1) or
           (self.current_player == 2 and self.p2_pits_index[0] <= index <= self.p2_pits_index[1] and self.board[index] == 1)):
                opposite_index = len(self.board) - 2 - index
                if self.board[opposite_index] >= 0:
                    self.board[self.p1_mancala_index if self.current_player == 1 else self.p2_mancala_index] += self.board[opposite_index] + 1
                    self.board[opposite_index] = 0
                    self.board[index] = 0
            self.current_player = 3 - self.current_player
        else:
            print("GAME OVER")
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                print("Player 1 Wins!")
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                print("Player 2 Wins!")
            else:
                print("It's a Tie!")
        
        return self.board

    def is_terminal(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_empty = all(self.board[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
        p2_empty = all(self.board[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))

        if p1_empty or p2_empty:
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                self.board[self.p1_mancala_index] += self.board[i]
                self.board[i] = 0
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                self.board[self.p2_mancala_index] += self.board[i]
                self.board[i] = 0
            return True

        return False


In [51]:
p1_wins = 0
p2_wins = 0
p1_loss = 0
p2_loss = 0
ties = 0
total_turns = 0
total_player_one_turns = 0
total_player_two_turns = 0

for game_num in range(100):
    
    game = Mancala()
    turns = 0
    amount_of_moves = 0
    player_one_turns = 0
    player_two_turns = 0
    turn_player_one = True

    while not game.is_terminal():
        move = game.random_move_generator()

        if move is None:
            game.is_terminal()
            break

        game.play(move)
        turns += 1
        if(turn_player_one):
            player_one_turns += 1
            turn_player_one = False
        else:
            player_two_turns += 1
            turn_player_one = True

        amount_of_moves += 1
        
        if amount_of_moves > 300:
            break

    total_turns += turns
    total_player_one_turns += player_one_turns
    total_player_two_turns += player_two_turns

    p1_score = game.board[game.p1_mancala_index]
    p2_score = game.board[game.p2_mancala_index]

    if p1_score > p2_score:
        p1_wins += 1
        
    elif p2_score > p1_score:
        p2_wins += 1
        
    else:
        ties += 1

p1_loss = 100 - p1_wins - ties
p2_loss = 100 - p2_wins - ties

print(f"Player 1 Wins: {p1_wins} ({p1_wins}%)")
print(f"Player 1 Losses: {p1_loss} ({p1_loss}%)")
print(f"Player 1 Ties: {ties} ({ties}%)")
print(f"Player 2 Wins: {p2_wins} ({p2_wins}%)")
print(f"Player 2 Losses: {p2_loss} ({p2_loss}%)")
print(f"Player 2 Ties: {ties} ({ties}%)")
print(f"Average number of turns per game: {total_turns / 100:.2f}")
print(f"Average number of turns per game for Player One: {total_player_one_turns / 100:.2f}")
print(f"Average number of turns per game for Player Two: {total_player_two_turns / 100:.2f}")

Player 1 Wins: 50 (50%)
Player 1 Losses: 47 (47%)
Player 1 Ties: 3 (3%)
Player 2 Wins: 47 (47%)
Player 2 Losses: 50 (50%)
Player 2 Ties: 3 (3%)
Average number of turns per game: 45.28
Average number of turns per game for Player One: 22.88
Average number of turns per game for Player Two: 22.40


## Project Writeup

### Is there a first move advantage? If so, how much?

In this case, there is a first move advantage. The statistics reported here tell us that there is about a 3-5% higher chance of winning if the player goes first than second.

### Current Status of Code and Project

From the code above, we have successfully implemented the Game class from the aima library games4e to include the Game class in the Mancala game itself. We have also successfully implemented a random player that is able to choose from a list of valid moves to play, and we have also extended this to be able to play 100 games of a random player vs a random player to show the statistics of what randomness looks like in this game. From the results above, we have determined that each player wins roughly 50% of the time, with a small number of ties (3%). On average, it takes about 23 moves for a player to win the game. We have also implemented all of the rules of the Mancala game as described in the Project Guidelines.

### List of Libraries and Frameworks Used

* aima libraries games4e and utilities4e
* From the games4e library, we imported the class framework Game
* HW 6 framework for the complete Mancala game

## Using MiniMax and Alpha-Beta Algorithms

In [52]:
class MancalaGame(Game):
    def __init__(self, pits_per_player=6, stones_per_pit=4):
        self.pits_per_player = pits_per_player
        self.total_pits = 2 * (pits_per_player + 1)
        self.p1_pits = list(range(pits_per_player))
        self.p1_mancala = pits_per_player
        self.p2_pits = list(range(pits_per_player + 1, self.total_pits - 1))
        self.p2_mancala = self.total_pits - 1

        board = [stones_per_pit] * self.total_pits
        board[self.p1_mancala] = 0
        board[self.p2_mancala] = 0

        self.initial = GameState(
            to_move=1,
            utility=0,
            board=board,
            moves=self.get_legal_moves(board, 1)
        )

    def get_legal_moves(self, board, player):
        if player == 1:
            return [i + 1 for i in self.p1_pits if board[i] > 0]
        else:
            return [i - self.pits_per_player for i in self.p2_pits if board[i] > 0]

    def actions(self, state):
        return self.get_legal_moves(state.board, state.to_move)

    def result(self, state, action):
        board = deepcopy(state.board)
        player = state.to_move
        pit_index = (action - 1 if player == 1 else self.p2_pits[0] + (action - 1))

        stones = board[pit_index]
        board[pit_index] = 0
        index = pit_index
        
        while stones > 0:
            index = (index + 1) % len(board)
            if (player == 1 and index == self.p2_mancala) or (player == 2 and index == self.p1_mancala):
                continue
            else:
                board[index] += 1
                stones -= 1

        if player == 1 and index in self.p1_pits and board[index] == 1:
            opposite_index = self.p2_pits[-(index + 1)]
            board[self.p1_mancala] += board[opposite_index] + 1
            board[opposite_index] = 0
            board[index] = 0
            
        elif player == 2 and index in self.p2_pits and board[index] == 1:
            opposite_index = self.p1_pits[-(index - self.p2_pits[0])]
            board[self.p2_mancala] += board[opposite_index] + 1
            board[opposite_index] = 0
            board[index] = 0

        is_terminal = self.terminal_test(board)
        utility = 0
        
        if is_terminal:
            self.collect_remaining(board)
            utility = board[self.p1_mancala] - board[self.p2_mancala]

        return GameState(
            to_move=(2 if player == 1 else 1),
            utility=utility,
            board=board,
            moves=self.get_legal_moves(board, 2 if player == 1 else 1)
        )

    def utility(self, state, player):
        
        if player == 1:
            player_mancala = self.p1_mancala
            player_pits = self.p1_pits
            opposing_player_mancala = self.p2_mancala
            opposing_player_pits = self.p2_pits
        else:
            player_mancala = self.p2_mancala
            player_pits = self.p2_pits
            opposing_player_mancala = self.p1_mancala
            opposing_player_pits = self.p1_pits

        score_of_player = state.board[player_mancala] + sum(state.board[i] for i in player_pits)
        score_of_opposing_player = state.board[opposing_player_mancala] + sum(state.board[i] for i in opposing_player_pits)

        return score_of_player - score_of_opposing_player

    def terminal_test(self, board):
        return (all(board[i] == 0 for i in self.p1_pits) or
                all(board[i] == 0 for i in self.p2_pits))

    def to_move(self, state):
        return state.to_move

    def collect_remaining(self, board):
        for i in self.p1_pits:
            board[self.p1_mancala] += board[i]
            board[i] = 0
        for i in self.p2_pits:
            board[self.p2_mancala] += board[i]
            board[i] = 0
        return board


### Minimax Player vs Random Player

In [54]:
from games4e import GameState, minmax_decision

p1_wins = 0
p2_wins = 0
ties = 0
p1_total_turns = 0
p2_total_turns = 0

for game_num in range(100):
    game = Mancala()
    p1_turns = 0
    p2_turns = 0
    total_turns = 0

    while not game.is_terminal():
        
        
        if game.current_player == 1:
            search_game = MancalaGame()
            search_game.initial = GameState(
                to_move=1,
                utility=0,
                board=game.board[:],
                moves=search_game.get_legal_moves(game.board, 1)
            )

            move = minmax_decision(search_game.initial, search_game, depth=5)
            p1_turns += 1
        else:
            move = game.random_move_generator()
            if move is None:
                game.is_terminal()
                break
            p2_turns += 1

        game.play(move)
        total_turns += 1
        if total_turns > 300:
            break

    p1_score = game.board[game.p1_mancala_index]
    p2_score = game.board[game.p2_mancala_index]

    if p1_score > p2_score:
        p1_wins += 1
    elif p2_score > p1_score:
        p2_wins += 1
    else:
        ties += 1

    p1_total_turns += p1_turns
    p2_total_turns += p2_turns

print("\n Player 1 Minmax")
print(f"Wins: {p1_wins} ({p1_wins}%)")
print(f"Losses: {p2_wins} ({p2_wins}%)")
print(f"Ties: {ties} ({ties}%)")
print(f"Average turns per game: {p1_total_turns / 100:.2f}")

print("\nPlayer 2 Random")
print(f"Wins: {p2_wins} ({p2_wins}%)")
print(f"Losses: {p1_wins} ({p1_wins}%)")
print(f"Ties: {ties} ({ties}%)")
print(f"Average turns per game: {p2_total_turns / 100:.2f}")


 Player 1 Minmax
Wins: 90 (90%)
Losses: 8 (8%)
Ties: 2 (2%)
Average turns per game: 16.88

Player 2 Random
Wins: 8 (8%)
Losses: 90 (90%)
Ties: 2 (2%)
Average turns per game: 24.86


### Alpha Beta Player vs Random Player

In [58]:
p1_wins = 0
p2_wins = 0
ties = 0
p1_total_turns = 0
p2_total_turns = 0
total_time = 0

for game_num in range(100):
    game = Mancala()
    p1_turns = 0
    p2_turns = 0
    total_turns = 0
    startTime = time.time()
    while not game.is_terminal():
        
        if game.current_player == 1:
            search_game = MancalaGame()
            search_game.initial = GameState(
                to_move=1,
                utility=0,
                board=game.board[:],
                moves=search_game.get_legal_moves(game.board, 1)
            )

            move = alpha_beta_search(search_game.initial, search_game, depth=5)
            p1_turns += 1
        else:
            move = game.random_move_generator()
            if move is None:
                game.is_terminal()
                break
            p2_turns += 1

        game.play(move)
        total_turns += 1
        elapsedTime = time.time() - startTime
        if total_turns > 300:
            break

    p1_score = game.board[game.p1_mancala_index]
    p2_score = game.board[game.p2_mancala_index]
    total_time += elapsedTime

    if p1_score > p2_score:
        p1_wins += 1
    elif p2_score > p1_score:
        p2_wins += 1
    else:
        ties += 1

    p1_total_turns += p1_turns
    p2_total_turns += p2_turns

print("\n Player 1 Alpha-Beta")
print(f"Wins: {p1_wins} ({p1_wins}%)")
print(f"Losses: {p2_wins} ({p2_wins}%)")
print(f"Ties: {ties} ({ties}%)")
print(f"Average turns per game: {p1_total_turns / 100:.2f}")

print("\nPlayer 2 Random")
print(f"Wins: {p2_wins} ({p2_wins}%)")
print(f"Losses: {p1_wins} ({p1_wins}%)")
print(f"Ties: {ties} ({ties}%)")
print(f"Average turns per game: {p2_total_turns / 100:.2f}")

print(f"\nTime taken for 1 game to run: {total_time / 100}")


 Player 1 Alpha-Beta
Wins: 88 (88%)
Losses: 10 (10%)
Ties: 2 (2%)
Average turns per game: 17.24

Player 2 Random
Wins: 10 (10%)
Losses: 88 (88%)
Ties: 2 (2%)
Average turns per game: 30.25

Time taken for 1 game to run: 0.02317600727081299


## Writeup for Final Part of the Project

#### Minimax vs Random Player

Given our code, we have run 100 games using a MiniMax player against a Random Player. The results show that 90% of the time, the AI player will win with 3 ties and 7 losses. On the other hand, the Random Player performs at a significantly worse level with only 7 wins and 90 losses, along with 3 ties. On average, It takes the AI player around 18 turns to win the game. The Random Player, on average, takes about 17 turns to win as well. 

Here, the AI performs better than the Random Player due to the algorithm that MiniMax uses to choose the best possible move. Since the MiniMax player is the Max player, it chooses a move such that the Random Player will be at a disadvantage when the game enters a terminal state. Rather than randomly choosing a pit to take stones from, there is a choice made that will maximize the advantage for the MiniMax player. As seen with the statistics, the AI player wins 90% of the time, which, when compared to the Random Player vs Random Player game, shows that the AI player wins far more than 50% of the games. 

#### AlphaBeta vs Random Player

We have run 100 games using a Alpha Beta player against a Random Player. The results show that 88% of the time, the AI player will win with 2 ties and 10 losses. On the other hand, the Random Player performs, once again, at a significantly worse level with only 3 wins and 96 losses, along with 1 tie. On average, It takes the AI player around 18 turns to win the game as well as the Random Player to win with 30 moves. 

In this case, the AI player once again performs better than the Random Player due to the Algorithm of Alpha-Beta Pruning. Since it is a version of MiniMax, with the only difference being that nodes that are not considered will be pruned, it once again chooses the move that will maximize the advantage for the Max player (i.e. the AI player) while simultaneously putting the Min player (Random Player) at a disadvantage when choosing a move. A game, on average, takes about 0.02 seconds to run using this algorithm. In comparison to the MiniMax algorithm, it can be seen that Alpha Beta performs worse with about a 2% decrease in performance, most likely due to pruning decisions as opposed to looking at all possible choices that might give the AI player an advantage. 

## Extra Credit

In [60]:
p1_wins = 0
p2_wins = 0
ties = 0
p1_total_turns = 0
p2_total_turns = 0
total_time = 0

for game_num in range(100):
    game = Mancala()
    p1_turns = 0
    p2_turns = 0
    total_turns = 0
    startTime = time.time()
    while not game.is_terminal():
        
        if game.current_player == 1:
            search_game = MancalaGame()
            search_game.initial = GameState(
                to_move=1,
                utility=0,
                board=game.board[:],
                moves=search_game.get_legal_moves(game.board, 1)
            )

            move = alpha_beta_search(search_game.initial, search_game, depth=10)
            p1_turns += 1
        else:
            move = game.random_move_generator()
            if move is None:
                game.is_terminal()
                break
            p2_turns += 1

        game.play(move)
        total_turns += 1
        elapsedTime = time.time() - startTime
        if total_turns > 300:
            break

    p1_score = game.board[game.p1_mancala_index]
    p2_score = game.board[game.p2_mancala_index]
    total_time += elapsedTime

    if p1_score > p2_score:
        p1_wins += 1
    elif p2_score > p1_score:
        p2_wins += 1
    else:
        ties += 1

    p1_total_turns += p1_turns
    p2_total_turns += p2_turns

print("\n Player 1 Alpha-Beta")
print(f"Wins: {p1_wins} ({p1_wins}%)")
print(f"Losses: {p2_wins} ({p2_wins}%)")
print(f"Ties: {ties} ({ties}%)")
print(f"Average turns per game: {p1_total_turns / 100:.2f}")

print("\nPlayer 2 Random")
print(f"Wins: {p2_wins} ({p2_wins}%)")
print(f"Losses: {p1_wins} ({p1_wins}%)")
print(f"Ties: {ties} ({ties}%)")
print(f"Average turns per game: {p2_total_turns / 100:.2f}")

print(f"\nTime taken for 1 game to run: {total_time / 100}")


 Player 1 Alpha-Beta
Wins: 99 (99%)
Losses: 1 (1%)
Ties: 0 (0%)
Average turns per game: 16.89

Player 2 Random
Wins: 1 (1%)
Losses: 99 (99%)
Ties: 0 (0%)
Average turns per game: 41.27

Time taken for 1 game to run: 2.4188065409660338


It takes about 2.41 sconds to run a single game to completion. The AI player with Alpha-Beta wins 99% of the games while the Random player only wins 1% of games. It takes about 16 moves for the AI player to win a game.

This new utility function looks at all aspects of future gameplay instead of just analyzing terminal states. For the original utility function, it simply looks at the score at that moment without considering the amount of stones that are still on the board. With our new utility function, we are able to analyze future potential alongside the score at that moment. This is through looking at the landscape of the board and analyzing the positions of the stones that are in the pits alongside working with the number of stones in the Mancala. With this, the AI is able to be more calculating and strategic instead of simply using the current Mancala score to find its next move. Also, the AI is less likely to make moves that focus on getting little gains into the Mancala at the expense of long-term strategies that can get more stones into the Mancala if played correctly.

As we increase the number of plies to 10, we are allowing our AI to view deeper into our game tree. With this, the AI is able to view a lot further down the line and see how a certain move can impact the game many turns later. Incrasing the amount of piles works to help our AI have a better long-term strategy and implement moves that are designed to build towards a much larger goal than trying to find short-term gains when we can. That is the issue with working at smaller depths as the AI can appear more greedy and simply do moves that help at the time but do not have much thought in them towards having an intricate strategy to win the game.

The new utility function is a better way to evaluate the strength of a particular match. We spoke about this a bit in the first question above but this is because we are considering the current score and the future potential for scoring. Looking at the original utility function, we are only looking at the stones that are in the Mancala. With this new utility function, we are considering the stones that are in the pits and are still in play. As we consider the positions of the stones that are not in the Mancala, we are able to create a much better strategy in terms of controlling the board and making moves that take advantage of the current positioning as posed to simply looking at short-term gains by increasing the amount of stones in the Mancala.